In [1]:
import numpy as np
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [8]:
def reformat(tens):
    return tens[0].T[0]

In [2]:
def int2bin(i, length=10):
    b = bin(i)[2:].zfill(length)
    b_lst = [int(i) for i in b]
    b_lst.reverse()
    return np.array(b_lst)
#     return np.array([int(i) for i in b])
#     return np.array([int(i) for i in b])

def bin2in(s):
    return int(s, 2)

In [5]:
def gen_data():
    max_val = 100
    A = np.random.randint(max_val)
    B = np.random.randint(max_val)
    C = A+B
    Ab = int2bin(A)
    Bb = int2bin(B)
    Cb = int2bin(C)
        
    X_batch = None
    X_batch = np.vstack((Ab, Bb)).T
    X_batch = X_batch[np.newaxis, :]
    y_batch = Cb[np.newaxis, :, np.newaxis]
    
    return X_batch, y_batch

def gen_data_batch(length):
    bin_len = 10
    
    A_lst = [int(i) for i in list(np.random.rand(length)*100)]
    B_lst = [int(i) for i in list(np.random.rand(length)*100)]
    C_lst = []
    
    X = np.zeros((length, bin_len, 2))
    Y = np.zeros((length, bin_len, 1))
    
    for idx, (a, b) in enumerate(zip(A_lst, B_lst)):
        c = a+b
        
        Ab = int2bin(a, bin_len)
        Bb = int2bin(b, bin_len)
        Cb = int2bin(c, bin_len)
        
        X[idx] = np.vstack((Ab, Bb)).T
        Y[idx] = Cb[:, np.newaxis]
        
    return X, Y

In [54]:
reset_graph()

n_steps   = 10
n_inputs  = 2
n_outputs = 1
n_neurons = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

sm = outputs
# sm = tf.nn.sigmoid(outputs)

init = tf.global_variables_initializer()

In [55]:
# learning_rate = 0.0001
learning_rate = 0.01

loss = tf.reduce_mean(tf.square(sm-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [6]:
# X_all, Y_all = gen_data_batch(100000)

In [7]:
# X_batch, y_batch = gen_data()
# print(X_batch[0].T[0])
# print(X_batch[0].T[1])
# print("sum")
# print(y_batch[0].T[0])

[1 1 1 0 0 0 1 0 0 0]
[0 0 0 1 0 1 0 0 0 0]
sum
[1 1 1 1 0 1 1 0 0 0]


In [56]:
# n_iterations = 60000
batch_size   = 50
n_epochs     = 30

saver = tf.train.Saver()

n_examples = 10000

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for iteration in range(n_examples // batch_size):
            X_batch, y_batch = gen_data_batch(batch_size)
        
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

            if iteration % 1000 == 0:
                mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
                print(iteration, "\tMSE:", mse)

#         break
    saver.save(sess, "./model")

0 	MSE: 0.315312
0 	MSE: 0.218505
0 	MSE: 0.207861
0 	MSE: 0.174188
0 	MSE: 0.115172
0 	MSE: 0.115688
0 	MSE: 0.10996
0 	MSE: 0.125371
0 	MSE: 0.1221
0 	MSE: 0.0946702
0 	MSE: 0.113816
0 	MSE: 0.12596
0 	MSE: 0.11944
0 	MSE: 0.0969933
0 	MSE: 0.0887123
0 	MSE: 0.114526
0 	MSE: 0.0970739
0 	MSE: 0.100872
0 	MSE: 0.102859
0 	MSE: 0.101766
0 	MSE: 0.118263
0 	MSE: 0.0987474
0 	MSE: 0.0996745
0 	MSE: 0.0965063
0 	MSE: 0.104891
0 	MSE: 0.0950817
0 	MSE: 0.0931288
0 	MSE: 0.0857615
0 	MSE: 0.0963412
0 	MSE: 0.0903998


In [63]:
with tf.Session() as sess:
    saver.restore(sess, "./model")
    X_batch, y_batch = gen_data()
    res = sm.eval(feed_dict={X: X_batch, y: y_batch})
#     res = states.eval(feed_dict={X: X_batch, y: y_batch})
#     print(res)
    print(X_batch)
    res[res>0.5] = 1
    res[res<=0.5] = 0
    print(reformat(res))
    print(reformat(y_batch))
    
    print(reformat(res-y_batch))

INFO:tensorflow:Restoring parameters from ./model
[[[0 0]
  [0 1]
  [1 0]
  [1 1]
  [0 0]
  [1 0]
  [0 0]
  [0 0]
  [0 0]
  [0 0]]]
[ 0.  1.  1.  0.  1.  1.  0.  0.  0.  0.]
[0 1 1 0 1 1 0 0 0 0]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
